In [4]:
import qiskit
from qiskit import quantum_info
from qiskit.execute_function import execute
from qiskit import BasicAer
import numpy as np
import pickle
import json
import os
from collections import Counter
from sklearn.metrics import mean_squared_error
from typing import Dict, List

#define utility functions

#this simulate a circuit (without measurement) and output results in the format of histogram.
def simulate(circuit: qiskit.QuantumCircuit) -> dict:
    """Simulate the circuit, give the state vector as the result."""
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circuit, backend)
    result = job.result()
    state_vector = result.get_statevector()
    
    histogram = dict()
    for i in range(len(state_vector)):
        population = abs(state_vector[i]) ** 2
        if population > 1e-9:
            histogram[i] = population
    
    return histogram


def histogram_to_category(histogram):
    assert abs(sum(histogram.values())-1)<1e-8
    positive=0
    for key in histogram.keys():
        digits = bin(int(key))[2:].zfill(20)
        if digits[-1]=='0':
            positive+=histogram[key]
        
    return positive

def count_gates(circuit: qiskit.QuantumCircuit) -> Dict[int, int]:
    """Returns the number of gate operations with each number of qubits."""
    return Counter([len(gate[1]) for gate in circuit.data])

def image_mse(image1,image2):
    # Using sklearns mean squared error:
    # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
    return mean_squared_error(image1, image2)


In [5]:
#load data
files=os.listdir("mock_data")
dataset=list()
for file in files:
    with open('mock_data/'+file, "r") as infile:
        loaded = json.load(infile)
        dataset.append(loaded)

In [6]:
#grade part1
from part1 import encode_qiskit,decode

n=len(dataset)
mse=0
gatecount=0

for data in dataset:
    #encode image into circuit
    circuit=encode_qiskit(data['image'])
    
    #simulate circuit
    histogram=simulate(circuit)
    
    gatecount+=count_gates(circuit)[2]
    
    #reconstruct 
    image_re=decode(histogram)
    
    #calculate mse
    mse+=image_mse(np.asarray(data['image']),np.asarray(image_re))
    
#fidelity of reconstruction
f=1-mse
gatecount=gatecount/n

#score for part1
print(f*(0.999**gatecount))

ImportError: cannot import name 'encode_qiskit' from 'part1' (/home/marco/hackaton/2023_IonQ_Remote/part1.py)

In [7]:
#grade part 2

#load classifier circuit
classifier=qiskit.QuantumCircuit.from_qasm_file('part2.qasm')
    
score=0

for data in dataset:
    #encode image into circuit
    circuit=encode_qiskit(data['image'])
    
    nq1 = circuit.width()
    nq2 = classifier.width()
    nq = max(nq1, nq2)
    qc = qiskit.QuantumCircuit(nq)
    qc.append(circuit.to_instruction(), list(range(nq1)))
    qc.append(classifier.to_instruction(), list(range(nq2)))

    gatecount+=count_gates(qc)[2]
    
    #simulate circuit
    histogram=simulate(qc)
    
    #convert histogram to category
    cat=histogram_to_category(histogram)
    
    
    if cat==data['category']:
        score+=1
#score
score=score/n
gatecount=gatecount/n

print(score*(0.999**gatecount))

NameError: name 'encode_qiskit' is not defined